In [149]:
from Bio import SeqIO
from Bio.Seq import MutableSeq

read = SeqIO.read('databases/APOE2.fna', 'fasta')
read.seq

Seq('CTACTCAGCCCCAGCGGAGGTGAAGGACGTCCTTCCCCAGGAGCCGGTGAGAAG...GCA')

In [150]:
seq_sl = read.seq[0:20]
seq_sl

Seq('CTACTCAGCCCCAGCGGAGG')

In [151]:
# Original
mutable_seq = MutableSeq(seq_sl)
mem = mutable_seq.transcribe().translate()
mem

MutableSeq('LLSPSG')

#### Silent Mutation

In [152]:
mutable_seq[0] = "T"
mutable_seq

MutableSeq('TTACTCAGCCCCAGCGGAGG')

In [153]:
mem2 = mutable_seq.transcribe().translate()
print('Original:\t', mem)
print('Silent:\t\t', mem2)

Original:	 LLSPSG
Silent:		 LLSPSG


Изменили один нуклеотид, мутация прошла не замеченной (аминокислотная последовательность белка осталась неизменной).

#### Nonsense

Для получения Nonsense мутации мы должны получить один из трех стоп-кодонов: UAA, UAG, UGA.

In [154]:
mutable_seq = MutableSeq(seq_sl)
mutable_seq

MutableSeq('CTACTCAGCCCCAGCGGAGG')

In [155]:
mutable_seq[3:6] = 'TAA' # convert to UAA
mutable_seq

MutableSeq('CTATAAAGCCCCAGCGGAGG')

In [156]:
mem3 = mutable_seq.transcribe().translate()
print('Original:\t', mem)
print('Nonsense:\t', mem3)

Original:	 LLSPSG
Nonsense:	 L*SPSG


#### Missense

Должны получить измененную последовательность.

In [157]:
mutable_seq = MutableSeq(seq_sl)
mutable_seq

MutableSeq('CTACTCAGCCCCAGCGGAGG')

In [160]:
mutable_seq[1] = 'C'
mutable_seq

MutableSeq('ACACTCAGCCCCAGCGGAGG')

In [161]:
mem3 = mutable_seq.transcribe().translate()

In [162]:
print('Original:\t', mem)
print('Missense:\t', mem3)

Original:	 LLSPSG
Missense:	 TLSPSG


# SNP

In [165]:
import sys
import time
from Bio import Entrez

In [173]:
Entrez.email = "omenbestg@gmail.com" # provide your user email
# RECOMMENDED: apply for API key from NCBI (https://ncbiinsights.ncbi.nlm.nih.gov/2017/11/02/new-api-keys-for-the-e-utilities/).
# 10 queries per second with a valid API key, otherwise 3 queries per seconds are allowed for 'None'
Entrez.api_key = None

# dbSNP supported query terms (https://www.ncbi.nlm.nih.gov/snp/docs/entrez_help/) can be build and test online using web query builder (https://www.ncbi.nlm.nih.gov/snp/advanced)
# esearch handle
eShandle = Entrez.esearch(db="snp",  # search dbSNP
                          #complex query for missense and pathogenic variants in LPL gene with global MAF betweeen 0 and 0.01.
                          term='348[GeneID] AND pathogenic[Clinical_Significance] AND missense variant[Function_Class] AND (00000.0000[GLOBAL_MAF] : 00000.0100[GLOBAL_MAF])',
                          usehistory="y", #cache result on server for download in batches
                          retmax=20 # return 20 RSID max
                         )

In [174]:
# get esearch result
eSresult = Entrez.read(eShandle)

In [175]:
# review results
for k in eSresult:
    print (k, ":", eSresult[k])

Count : 8
RetMax : 8
RetStart : 0
QueryKey : 1
WebEnv : MCID_65b41c0213f1a854cd561b9b
IdList : ['201672011', '199768005', '190853081', '140808909', '121918393', '121918392', '769455', '769452']
TranslationSet : []
TranslationStack : [{'Term': '348[GeneID]', 'Field': 'GeneID', 'Count': '2862', 'Explode': 'N'}, {'Term': 'pathogenic[Clinical_Significance]', 'Field': 'Clinical_Significance', 'Count': '130269', 'Explode': 'N'}, 'AND', {'Term': 'missense variant[Function_Class]', 'Field': 'Function_Class', 'Count': '10256260', 'Explode': 'N'}, 'AND', {'Term': '00000.0000[GLOBAL_MAF]', 'Field': 'GLOBAL_MAF', 'Count': '0', 'Explode': 'N'}, {'Term': '00000.0100[GLOBAL_MAF]', 'Field': 'GLOBAL_MAF', 'Count': '0', 'Explode': 'N'}, 'RANGE', 'GROUP', 'AND']
QueryTranslation : 348[GeneID] AND pathogenic[Clinical_Significance] AND missense variant[Function_Class] AND (00000.0000[GLOBAL_MAF] : 00000.0100[GLOBAL_MAF])


In [176]:
# get result RSIDs list 'Idlist'
# total rs count
rslist = (eSresult['IdList'])

In [177]:
# retmax = 20 so print only 20 RSIDs
# additional results can be retrieved by batches
# download in batches example http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc139 or see below.
for rs in rslist:
    print(rs)

201672011
199768005
190853081
140808909
121918393
121918392
769455
769452


In [178]:
# get the WebEnv session cookie, and the QueryKey:

webenv = eSresult["WebEnv"]
query_key = eSresult["QueryKey"]
total_count = int(eSresult["Count"])
query_key = eSresult["QueryKey"]
retmax = 2 # return 2 rs per batch example

In [180]:
# sample codes adopted with modifications from http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc139.
fetch_count = 0
for start in range(0, total_count, retmax):
    end = min(total_count, start+retmax)
    print("Going to download record %i to %i" % (start+1, end))
    attempt = 0
    #fetch_count += 1
    while (attempt < 3):
        attempt += 1
        try:
            fetch_handle = Entrez.efetch(db="snp",
                                         retmode="xml",
                                         retstart=start,
                                         retmax=retmax,
                                         webenv=webenv,
                                         query_key=query_key)
            break  # Добавьте это прерывание цикла после успешного выполнения запроса
        except HTTPError as err:
            if 500 <= err.code <= 599:
                print("Received error from server %s" % err)
                print("Attempt %i of 3" % attempt)
                time.sleep(15)
            else:
                raise
    if (fetch_handle):
        #print(fetch_handle)
        data = fetch_handle.read()
        print(data)
        fetch_handle.close()

Going to download record 1 to 2
b'<?xml version="1.0" ?>\n<ExchangeSet xmlns:xsi="https://www.w3.org/2001/XMLSchema-instance" xmlns="https://www.ncbi.nlm.nih.gov/SNP/docsum" xsi:schemaLocation="https://www.ncbi.nlm.nih.gov/SNP/docsum ftp://ftp.ncbi.nlm.nih.gov/snp/specs/docsum_eutils.xsd" ><DocumentSummary uid="201672011"><SNP_ID>201672011</SNP_ID><ALLELE_ORIGIN/><GLOBAL_MAFS><MAF><STUDY>1000Genomes</STUDY><FREQ>A=0.000625/3</FREQ></MAF><MAF><STUDY>ExAC</STUDY><FREQ>A=0.000159/18</FREQ></MAF><MAF><STUDY>GnomAD</STUDY><FREQ>A=0.000492/69</FREQ></MAF><MAF><STUDY>GnomAD_exomes</STUDY><FREQ>A=0.00012/30</FREQ></MAF><MAF><STUDY>PAGE_STUDY</STUDY><FREQ>A=0.001513/119</FREQ></MAF><MAF><STUDY>TOMMO</STUDY><FREQ>A=0.000106/2</FREQ></MAF><MAF><STUDY>TOPMED</STUDY><FREQ>A=0.001024/271</FREQ></MAF><MAF><STUDY>ALFA</STUDY><FREQ>A=0.00016/2</FREQ></MAF></GLOBAL_MAFS><GLOBAL_POPULATION/><GLOBAL_SAMPLESIZE>0</GLOBAL_SAMPLESIZE><SUSPECTED/><CLINICAL_SIGNIFICANCE>likely-benign,pathogenic</CLINICAL_SIGNI

In [181]:
# Печать количества RSID
print("Total number of RSIDs:", len(rslist))

Total number of RSIDs: 8
